In [29]:
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np
import matplotlib.pyplot as plt
import csv
%matplotlib inline

hijae = pd.read_csv("hijaedata.csv")
puree = pd.read_csv("pureedata.csv")[:-1]
hikari= pd.read_csv("hikaridata.csv")[:-1]

def addindex(dfr):
    index_col=[]
    for i in range(0, len(dfr)/5):
        index_col.append(i)
        index_col.append(i)
        index_col.append(i)
        index_col.append(i)
        index_col.append(i)
    dfr['index']=index_col


    
addindex(hijae)
addindex(hikari)
addindex(puree)
#df.head(10)
#df.tail(15)
#hikari.head()

In [36]:
#partitioned=df[:].ix[:,5:]
#summed=partitioned.groupby('index').sum()
#print summed.head()
#print len(summed.columns)
#아 이래서 클래스를 쓰는구나...;;

def hijaeframe(dfr):
    ##groupby index and sum
    partitioned=dfr.iloc[::,5:]
    summed=partitioned.groupby('index').sum()
    
    #final dataframe
    loldf= pd.DataFrame()
    loldf['winner']=[dfr['winner'][i] for i in range(0,len(dfr), 5)]
    
    #side - dummy variable
    side_dummy = pd.get_dummies([dfr['side_1'][i] for i in range(1,len(dfr), 5)], prefix='side')
    loldf[[i for i in side_dummy.columns]] = side_dummy
    
    loldf[[i for i in summed.columns]]=summed[[i for i in summed.columns]]
    loldf['newwin']=(loldf['win']/loldf['game_cnt'])*100
    loldf['topsin']=loldf['tpg']*(loldf['kpg']+loldf['apg'])
    loldf['ggg']=( (loldf['ddpg']-(loldf['cs']*750))/ (loldf['dpg']))
    loldf['kapg']=loldf['kpg']+loldf['apg']
    return loldf

model_blue = hijaeframe(hijae).iloc[1::2, :]
model_blue.to_csv("hijae_red.csv")
#model_blue.head()

#hijaeframe(puree).iloc[1::2, :].to_csv("puree_red.csv")
#hijaeframe(hikari).iloc[1::2, :].to_csv("hikari_red.csv")


In [37]:
def hijaelogr(dfr):
    logr = sm.Logit(dfr['winner'], dfr[['tpg', 'dpg', 'win_rate', 'topsin', 'kapg']])
    return logr

###  make this into class-objects ###
### 
hijaem = hijaelogr(model_blue).fit()
print hijaem.summary()
print np.exp(hijaem.params)

#print np.exp(hijaem.conf_int())

Optimization terminated successfully.
         Current function value: 0.467415
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:                 winner   No. Observations:                  400
Model:                          Logit   Df Residuals:                      395
Method:                           MLE   Df Model:                            4
Date:                Mon, 28 Mar 2016   Pseudo R-squ.:                  0.3202
Time:                        14:23:54   Log-Likelihood:                -186.97
converged:                       True   LL-Null:                       -275.05
                                        LLR p-value:                 4.960e-37
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
tpg           -2.9127      0.490     -5.948      0.000        -3.873    -1.953
dpg            0.0256      0.

In [19]:
hikari_blue = hijaeframe(hikari).iloc[::2, :]

hikari_blue.to_csv("hikari_blue.csv")


#a=hikari_blue[['newwin','topsin','kpg','dpg','apg','ggg']]
#print a.head(5)
#print np.exp(hijaem.params)
#print (np.exp(hijaem.params) * model_blue).head(5)
#print hijaem.predict()[:5]



In [110]:
######################################################
c=pd.DataFrame()
c['pred']=hijaem.predict(a)
c['winner']=[z for z in hikari_blue['winner']]

d=pd.DataFrame()
d['pred']=hijaem.predict()
d['winner']=[z for z in model_blue['winner']]
d.to_csv("d_result.csv")
